# 1. Scrap Site www.topuniversities.com

found the json using postman:

https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508252855868


In [7]:
#json to dataframe using panda
pass


# 2. Scrap Site www.timeshighereducation.com

found the json using postman again:

https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json

In [12]:
#json to dataframe using panda
pass




# 3. merge stuffs

In [13]:
# 2 dataframe to 1
pass





# 4. Do correlation?

In [14]:
#work
pass





# 5. Best university? (not EPFL)

In [15]:
#harward or stuff
pass


